# Spider Evaluation Matrix Results

## Spider All Experiments Evaluation Matrix

In [1]:
import json
import os
import glob
from tqdm import tqdm
from utils.utils import sql_similarity
from utils.utils import mask_query

pattern = os.path.join('chapter-3', 'results', 'spider', '**', '*results-1.json')

# Retrieve all matching file paths
results_files = glob.glob(pattern, recursive=True)
results_files.remove(os.path.join('chapter-3', 'results', 'spider', 'random-experiments', '1-shot', 'random-results-1.json'))


def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load all result data
matrix = [0,0,0,0]
results_data = {os.path.basename(file_path): load_json(file_path).get('questions', []) for file_path in results_files}

# Collect all questions into a single list
all_questions = []
for questions in results_data.values():
    all_questions.extend(questions)

for question in tqdm(all_questions):
    correct = question.get('correct')
    ex_sql = question['examples'][0]['query']
    gold_sql = 'SELECT ' + question['response']
    
    similarity = sql_similarity(mask_query(ex_sql), mask_query(gold_sql))
    #print(ex_sql,"/", gold_sql, similarity, "\n")
    
    if correct == 1:
        if similarity >= 0.75:
            matrix[0] += 1
        else:
            matrix[1] += 1
    elif correct == 0:
        if similarity > 0.75:
            matrix[2] += 1
        else:
            matrix[3] += 1

print(matrix)

100%|██████████| 8272/8272 [02:46<00:00, 49.78it/s] 

[2268, 4046, 277, 1681]


## Spider Average SQLSim(ex, gold) Scores

In [10]:
import json
import os
import glob
from tqdm import tqdm
from utils.utils import sql_similarity, mask_query
import numpy as np

pattern = os.path.join('chapter-3', 'results', 'spider', '**', '*results-1.json')

# Retrieve all matching file paths
results_files = glob.glob(pattern, recursive=True)
results_files.remove(os.path.join('chapter-3', 'results', 'spider', 'random-experiments', '1-shot', 'random-results-1.json'))

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load all result data
results_data = {os.path.basename(file_path): load_json(file_path).get('questions', []) for file_path in results_files}

# Collect all questions into a single list
all_questions = []
for questions in results_data.values():
    all_questions.extend(questions)

# Filter out questions where correct = 0
incorrect_questions = [question for question in all_questions if question.get('correct') == 0]

# Verify the number of incorrect questions
print(f"Number of questions where correct = 0: {len(incorrect_questions)}")

# Initialize the list to store sql_similarity scores for questions where correct = 0
similarity_scores = []

# Loop through all incorrect questions and measure sql_similarity
for question in tqdm(incorrect_questions):
    ex_sql = question['examples'][0]['query']
    gold_sql = 'SELECT ' + question['response']
    
    similarity = sql_similarity(mask_query(ex_sql), mask_query(gold_sql))
    similarity_scores.append(similarity)

# Compute the mean average sql_similarity score
mean_similarity = np.mean(similarity_scores)

print(f"Mean average sql_similarity score for questions where correct = 0: {mean_similarity}")

# ---------------------------------------------------------------------------------------------------------

# Filter out questions where correct = 1
correct_questions = [question for question in all_questions if question.get('correct') == 1]

# Verify the number of incorrect questions
print(f"Number of questions where correct = 1: {len(correct_questions)}")

# Initialize the list to store sql_similarity scores for questions where correct = 0
similarity_scores = []

# Loop through all incorrect questions and measure sql_similarity
for question in tqdm(correct_questions):
    ex_sql = question['examples'][0]['query']
    gold_sql = 'SELECT ' + question['response']
    
    similarity = sql_similarity(mask_query(ex_sql), mask_query(gold_sql))
    similarity_scores.append(similarity)

# Compute the mean average sql_similarity score
mean_similarity = np.mean(similarity_scores)

print(f"Mean average sql_similarity score for questions where correct = 1: {mean_similarity}")

Number of questions where correct = 0: 1958


100%|██████████| 1958/1958 [00:39<00:00, 49.86it/s]


Mean average sql_similarity score for questions where correct = 0: 0.4813593776326583
Number of questions where correct = 1: 6314


100%|██████████| 6314/6314 [01:07<00:00, 93.36it/s] 

Mean average sql_similarity score for questions where correct = 1: 0.6144154839161518


In [2]:
import json
import glob
import os
from utils.utils import mask_query, sql_similarity

def calculate_mean_example_quality(file_path):
    # Load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    total_quality = 0
    total_questions = len(data['questions'])
    
    for question in data['questions']:
        example_qualities = []
        for example in question['examples']:
            masked_query = 'SELECT' + mask_query(question['response'])
            masked_example_query = mask_query(example['query'])
            quality = sql_similarity(masked_query, masked_example_query)
            example_qualities.append(quality)
        
        mean_quality = sum(example_qualities) / len(example_qualities)
        total_quality += mean_quality
    
    overall_mean_quality = total_quality / total_questions
    return overall_mean_quality


# Retrieve all local json results-file paths from completed Spider runs.
pattern = os.path.join('chapter-3','results','spider', '**', '*results-[12345].json')
#
results = glob.glob(pattern, recursive=True)

# Tally up how many of the 1034 Spider questions have been answered correctly

for file_path in results:
    print(file_path, calculate_mean_example_quality(file_path))

chapter-3\results\spider\all-MiniLM-L12-v2-experiments\1-shot\all-MiniLM-L12-v2-results-1.json 0.5403660407648827
chapter-3\results\spider\all-MiniLM-L12-v2-experiments\3-shot\all-MiniLM-L12-v2-results-3.json 0.5275907423338566
chapter-3\results\spider\all-MiniLM-L12-v2-experiments\5-shot\all-MiniLM-L12-v2-results-5.json 0.5216673467770376
chapter-3\results\spider\all-MiniLM-L6-v2-experiments\1-shot\all-MiniLM-L6-v2-results-1.json 0.5269915784700737
chapter-3\results\spider\all-MiniLM-L6-v2-experiments\3-shot\all-MiniLM-L6-v2-results-3.json 0.5108134680056198
chapter-3\results\spider\all-MiniLM-L6-v2-experiments\5-shot\all-MiniLM-L6-v2-results-5.json 0.5069976256615013
chapter-3\results\spider\all-mpnet-base-v2-experiments\1-shot\all-mpnet-base-v2-results-1.json 0.5450070802597946
chapter-3\results\spider\all-mpnet-base-v2-experiments\3-shot\all-mpnet-base-v2-results-3.json 0.5251970974439
chapter-3\results\spider\all-mpnet-base-v2-experiments\5-shot\all-mpnet-base-v2-results-5.json 0.

## Spider % of Questions with a 'Good' Example

### 0.75 Threshold

In [3]:
from utils.utils import sql_similarity, mask_query
from tqdm import tqdm
from utils.data.data_builder import load_data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import euclidean_distances
import json
import numpy as np

path_data = "benchmarks"

data = load_data("spider", path_data)

embedding = SentenceTransformer('s2593817/sft-sql-embedding')

def read_sql_queries(file_path, has_db_name=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if has_db_name:
        queries = [line.split('\t')[0].strip() for line in lines]
    else:
        queries = [line.strip() for line in lines]
    return queries

def read_sql_queries_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return  [item['query'] for item in data]

def calculate_proportion(dev_file, train_gold_file, threshold=0.85):
    dev_queries = read_sql_queries(dev_file)
    train_gold_queries = read_sql_queries_json(train_gold_file)
    
    masked_train = [mask_query(query) for query in train_gold_queries]
    embeddings_train = embedding.encode(masked_train)
    
    matching_count = 0
    
    for dev_query in tqdm(dev_queries):
        
        # Firstly order candidates by fine-tuned SQL embedding distances
        masked_dev = mask_query(dev_query)
        embedding_dev = embedding.encode(masked_dev).reshape(1, -1)
        
        distances = np.squeeze(euclidean_distances(embedding_dev, embeddings_train)).tolist()
        pairs = [(distance, index) for distance, index in zip(distances, range(len(distances)))]

        pairs_sorted = sorted(pairs, key=lambda x: x[0])
        for d, index in pairs_sorted:
            # Determine the SQLSim value and if it is above the threshold, record a success and move on to the next question
            if sql_similarity(masked_dev, masked_train[index]) > 0.75:
                matching_count += 1
                break
    
    proportion = matching_count / len(dev_queries)
    return proportion

dev_file_path = 'benchmarks/spider/dev_gold.sql'
train_gold_file_path = 'benchmarks/spider/train_spider_and_others.json'

proportion = calculate_proportion(dev_file_path, train_gold_file_path)
print(f"Proportion of Spider test questions with a 'good' training example: {proportion:.2f}")

100%|██████████| 1034/1034 [4:59:14<00:00, 17.36s/it]   

Proportion of Spider test questions with a 'good' training example: 0.92


### 0.85 Threshold

In [5]:
from utils.utils import sql_similarity, mask_query
from tqdm import tqdm
from utils.data.data_builder import load_data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import euclidean_distances
import json
import numpy as np

path_data = "benchmarks"

data = load_data("spider", path_data)

embedding = SentenceTransformer('s2593817/sft-sql-embedding')

def read_sql_queries(file_path, has_db_name=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if has_db_name:
        queries = [line.split('\t')[0].strip() for line in lines]
    else:
        queries = [line.strip() for line in lines]
    return queries

def read_sql_queries_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return  [item['query'] for item in data]

def calculate_proportion(dev_file, train_gold_file, threshold=0.85):
    dev_queries = read_sql_queries(dev_file)
    train_gold_queries = read_sql_queries_json(train_gold_file)
    
    masked_train = [mask_query(query) for query in train_gold_queries]
    embeddings_train = embedding.encode(masked_train)
    
    matching_count = 0
    
    for dev_query in tqdm(dev_queries):
        
        # Firstly order candidates by fine-tuned SQL embedding distances
        masked_dev = mask_query(dev_query)
        embedding_dev = embedding.encode(masked_dev).reshape(1, -1)
        
        distances = np.squeeze(euclidean_distances(embedding_dev, embeddings_train)).tolist()
        pairs = [(distance, index) for distance, index in zip(distances, range(len(distances)))]

        pairs_sorted = sorted(pairs, key=lambda x: x[0])
        for d, index in pairs_sorted:
            # Determine the SQLSim value and if it is above the threshold, record a success and move on to the next question
            if sql_similarity(masked_dev, masked_train[index]) > 0.85:
                matching_count += 1
                break
    
    proportion = matching_count / len(dev_queries)
    return proportion

dev_file_path = 'benchmarks/spider/dev_gold.sql'
train_gold_file_path = 'benchmarks/spider/train_spider_and_others.json'

proportion = calculate_proportion(dev_file_path, train_gold_file_path)
print(f"Proportion of Spider test questions with a 'good' training example: {proportion:.2f}")

c:\Users\MDrap\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 1034/1034 [06:47<00:00,  2.54it/s]

Proportion of Spider test questions with a 'good' training example: 0.66


# BIRD Evaluation Matrix Results

## BIRD All Experiments Evaluation Matrix

In [1]:
import json
import os
import glob
from tqdm import tqdm
from utils.utils import sql_similarity
from utils.utils import mask_query

pattern = os.path.join('chapter-3', 'results', 'bird', '**', '*results-1.json')

# Retrieve all matching file paths
results_files = glob.glob(pattern, recursive=True)
results_files.remove(os.path.join('chapter-3', 'results', 'bird', 'random-experiments', '1-shot', 'random-results-1.json'))

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load all result data
matrix = [0,0,0,0]
results_data = {os.path.basename(file_path): load_json(file_path).get('questions', []) for file_path in results_files}

# Collect all questions into a single list
all_questions = []
for questions in results_data.values():
    all_questions.extend(questions)

for question in tqdm(all_questions):
    correct = question.get('correct')
    ex_sql = question['examples'][0]['query']
    gold_sql = 'SELECT ' + question['response']
    
    similarity = sql_similarity(mask_query(ex_sql), mask_query(gold_sql))
    #print(ex_sql,"/", gold_sql, similarity, "\n")
    
    if correct == 1:
        if similarity >= 0.75:
            matrix[0] += 1
        else:
            matrix[1] += 1
    elif correct == 0:
        if similarity > 0.75:
            matrix[2] += 1
        else:
            matrix[3] += 1

print(matrix)

100%|██████████| 12272/12272 [11:56<00:00, 17.14it/s] 

[767, 4533, 544, 6428]


## BIRD Average SQLSim(ex, gold) Scores

In [5]:
import json
import os
import glob
from tqdm import tqdm
from utils.utils import sql_similarity, mask_query
import numpy as np

pattern = os.path.join('chapter-3', 'results', 'bird', '**', '*results-1.json')

# Retrieve all matching file paths
results_files = glob.glob(pattern, recursive=True)
results_files.remove(os.path.join('chapter-3', 'results', 'bird', 'random-experiments', '1-shot', 'random-results-1.json'))

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load all result data
results_data = {os.path.basename(file_path): load_json(file_path).get('questions', []) for file_path in results_files}

# Collect all questions into a single list
all_questions = []
for questions in results_data.values():
    all_questions.extend(questions)

# Filter out questions where correct = 0
incorrect_questions = [question for question in all_questions if question.get('correct') == 0]

# Verify the number of incorrect questions
print(f"Number of questions where correct = 0: {len(incorrect_questions)}")

# Initialize the list to store sql_similarity scores for questions where correct = 0
similarity_scores = []

# Loop through all incorrect questions and measure sql_similarity
for question in tqdm(incorrect_questions):
    ex_sql = question['examples'][0]['query']
    gold_sql = 'SELECT ' + question['response']
    
    similarity = sql_similarity(mask_query(ex_sql), mask_query(gold_sql))
    similarity_scores.append(similarity)

# Compute the mean average sql_similarity score
mean_similarity = np.mean(similarity_scores)

print(f"Mean average sql_similarity score for questions where correct = 0: {mean_similarity}")

# ---------------------------------------------------------------------------------------------------------

# Filter out questions where correct = 1
correct_questions = [question for question in all_questions if question.get('correct') == 1]

# Verify the number of incorrect questions
print(f"Number of questions where correct = 1: {len(correct_questions)}")

# Initialize the list to store sql_similarity scores for questions where correct = 0
similarity_scores = []

# Loop through all incorrect questions and measure sql_similarity
for question in tqdm(correct_questions):
    ex_sql = question['examples'][0]['query']
    gold_sql = 'SELECT ' + question['response']
    
    similarity = sql_similarity(mask_query(ex_sql), mask_query(gold_sql))
    similarity_scores.append(similarity)

# Compute the mean average sql_similarity score
mean_similarity = np.mean(similarity_scores)

print(f"Mean average sql_similarity score for questions where correct = 1: {mean_similarity}")

Number of questions where correct = 0: 6972


100%|██████████| 6972/6972 [13:53<00:00,  8.36it/s]  


Mean average sql_similarity score for questions where correct = 0: 0.47058515484842145
Number of questions where correct = 1: 5300


100%|██████████| 5300/5300 [07:49<00:00, 11.30it/s]

Mean average sql_similarity score for questions where correct = 1: 0.505176271572087


In [3]:
import json
import glob
import os
from utils.utils import mask_query, sql_similarity

def calculate_mean_example_quality(file_path):
    # Load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    total_quality = 0
    total_questions = len(data['questions'])
    
    for question in data['questions']:
        example_qualities = []
        for example in question['examples']:
            masked_query = 'SELECT' + mask_query(question['response'])
            masked_example_query = mask_query(example['query'])
            quality = sql_similarity(masked_query, masked_example_query)
            example_qualities.append(quality)
        
        mean_quality = sum(example_qualities) / len(example_qualities)
        total_quality += mean_quality
    
    overall_mean_quality = total_quality / total_questions
    return overall_mean_quality


# Retrieve all local json results-file paths from completed Spider runs.
pattern = os.path.join('chapter-3','results','bird', '**', '*results-[12345].json')

results = glob.glob(pattern, recursive=True)

# Tally up how many of the 1034 Spider questions have been answered correctly

for file_path in results:
    print(file_path, calculate_mean_example_quality(file_path))

chapter-3\results\bird\all-MiniLM-L12-v2-experiments\1-shot\all-MiniLM-L12-v2-results-1.json 0.4704035114367873
chapter-3\results\bird\all-MiniLM-L12-v2-experiments\3-shot\all-MiniLM-L12-v2-results-3.json 0.4683763474837952
chapter-3\results\bird\all-MiniLM-L12-v2-experiments\5-shot\all-MiniLM-L12-v2-results-5.json 0.466927815324469
chapter-3\results\bird\all-MiniLM-L6-v2-experiments\1-shot\all-MiniLM-L6-v2-results-1.json 0.466358913494554
chapter-3\results\bird\all-MiniLM-L6-v2-experiments\3-shot\all-MiniLM-L6-v2-results-3.json 0.4666040378936047
chapter-3\results\bird\all-MiniLM-L6-v2-experiments\5-shot\all-MiniLM-L6-v2-results-5.json 0.46569540947537685
chapter-3\results\bird\all-mpnet-base-v2-experiments\1-shot\all-mpnet-base-v2-results-1.json 0.4797684322731596
chapter-3\results\bird\all-mpnet-base-v2-experiments\3-shot\all-mpnet-base-v2-results-3.json 0.48051126065769534
chapter-3\results\bird\all-mpnet-base-v2-experiments\5-shot\all-mpnet-base-v2-results-5.json 0.480562527221327

## BIRD % of Questions with a 'Good' Example

### 0.75 Threshold

In [3]:
from utils.utils import sql_similarity, mask_query
from tqdm import tqdm

def read_sql_queries(file_path, has_db_name=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if has_db_name:
        queries = [line.split('\t')[0].strip() for line in lines]
    else:
        queries = [line.strip() for line in lines]
    return queries

def calculate_proportion(dev_file, train_gold_file, threshold=0.75):
    dev_queries = read_sql_queries(dev_file, has_db_name=True)
    train_gold_queries = read_sql_queries(train_gold_file)
    
    masked_train = [mask_query(query) for query in train_gold_queries]
    embeddings_train = embedding.encode(masked_train)
    
    matching_count = 0
    
    for dev_query in tqdm(dev_queries):
        
        # Firstly order candidates by fine-tuned SQL embedding distances
        masked_dev = mask_query(dev_query)
        embedding_dev = embedding.encode(masked_dev).reshape(1, -1)
        
        distances = np.squeeze(euclidean_distances(embedding_dev, embeddings_train)).tolist()
        pairs = [(distance, index) for distance, index in zip(distances, range(len(distances)))]

        pairs_sorted = sorted(pairs, key=lambda x: x[0])
        for d, index in pairs_sorted:
            # Determine the SQLSim value and if it is above the threshold, record a success and move on to the next question
            if sql_similarity(masked_dev, masked_train[index]) > 0.75:
                matching_count += 1
                break
    
    proportion = matching_count / len(dev_queries)
    return proportion

dev_file_path = 'benchmarks/bird/dev.sql'
train_gold_file_path = 'benchmarks/bird/train_gold.sql'

proportion = calculate_proportion(dev_file_path, train_gold_file_path)
print(f"Proportion of BIRD test questions with a 'good' training example: {proportion:.2f}")

100%|██████████| 1534/1534 [37:35:56<00:00, 88.24s/it]      

Proportion of BIRD test questions with a 'good' training example: 0.88


### 0.85 Threshold

In [6]:
from utils.utils import sql_similarity, mask_query
from tqdm import tqdm

def read_sql_queries(file_path, has_db_name=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if has_db_name:
        queries = [line.split('\t')[0].strip() for line in lines]
    else:
        queries = [line.strip() for line in lines]
    return queries

def calculate_proportion(dev_file, train_gold_file, threshold=0.75):
    dev_queries = read_sql_queries(dev_file, has_db_name=True)
    train_gold_queries = read_sql_queries(train_gold_file)
    
    masked_train = [mask_query(query) for query in train_gold_queries]
    embeddings_train = embedding.encode(masked_train)
    
    matching_count = 0
    
    for dev_query in tqdm(dev_queries):
        
        # Firstly order candidates by fine-tuned SQL embedding distances
        masked_dev = mask_query(dev_query)
        embedding_dev = embedding.encode(masked_dev).reshape(1, -1)
        
        distances = np.squeeze(euclidean_distances(embedding_dev, embeddings_train)).tolist()
        pairs = [(distance, index) for distance, index in zip(distances, range(len(distances)))]

        pairs_sorted = sorted(pairs, key=lambda x: x[0])
        for d, index in pairs_sorted:
            # Determine the SQLSim value and if it is above the threshold, record a success and move on to the next question
            if sql_similarity(masked_dev, masked_train[index]) > 0.85:
                matching_count += 1
                break
    
    proportion = matching_count / len(dev_queries)
    return proportion

dev_file_path = 'benchmarks/bird/dev.sql'
train_gold_file_path = 'benchmarks/bird/train_gold.sql'

proportion = calculate_proportion(dev_file_path, train_gold_file_path)
print(f"Proportion of BIRD test questions with a 'good' training example: {proportion:.2f}")

100%|██████████| 1534/1534 [20:06<00:00,  1.27it/s]  

Proportion of BIRD test questions with a 'good' training example: 0.66
